# Extract data from Supplementary Excel files

The purpose of this notebook is to extract data from the Excel files, clean them, and write them to CSV.


In [1]:
import os
import common

# Assign notebook and folder names
notebook_name = '04_extract_data_from_supplementary_excel_files'
figure_folder = os.path.join(common.FIGURE_FOLDER, notebook_name)
data_folder = os.path.join(common.DATA_FOLDER, notebook_name)
print('Figure folder:', figure_folder)
print('Data folder:', data_folder)

# Make the folders
! mkdir -p $figure_folder
! mkdir -p $data_folder

Figure folder: ../figures/04_extract_data_from_supplementary_excel_files
Data folder: ../data/04_extract_data_from_supplementary_excel_files


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

In [3]:
input_folder = os.path.join(common.DATA_FOLDER, '00_original')

In [4]:
ls $input_folder

GSM1544798_SpeciesMix_ThousandSTAMPs_HUMAN.digital_expression.txt.gz
GSM1544798_SpeciesMix_ThousandSTAMPs_MOUSE.digital_expression.txt.gz
GSM1544799_SpeciesMix_HundredSTAMPs_HUMAN.digital_expression.txt.gz
GSM1544799_SpeciesMix_HundredSTAMPs_MOUSE.digital_expression.txt.gz
GSM1626793_P14Retina_1.digital_expression.txt.gz
GSM1626794_P14Retina_2.digital_expression.txt.gz
GSM1626795_P14Retina_3.digital_expression.txt.gz
GSM1626796_P14Retina_4.digital_expression.txt.gz
GSM1626797_P14Retina_5.digital_expression.txt.gz
GSM1626798_P14Retina_6.digital_expression.txt.gz
GSM1626799_P14Retina_7.digital_expression.txt.gz
GSM1629192_Pure_HumanMouse_HUMAN.digital_expression.txt.gz
GSM1629192_Pure_HumanMouse_MOUSE.digital_expression.txt.gz
GSM1629193_ERCC.digital_expression.txt.gz
GSM1629193_hg19_ERCC.dict.txt.gz
GSM1629193_hg19_ERCC.refFlat.txt.gz
mmc1.pdf
mmc2.xlsx
mmc3.xlsx
mmc4.xlsx
mmc4_v2.xlsx
retina_clusteridentities.txt
~$mmc4_v2.xlsx


## Cluster markers

These are the genes that were preferentially expressed in individual clusters, according to their analysis.


The original supplementary file was corrupted and had many problems with the column values so I fixed them by hand, which is why we use `mmc4_v2.xlsx`.

In [22]:
filename = os.path.join(input_folder, 'mmc4_v2.xlsx')

cluster_markers = pd.read_excel(filename,
                                sheetname='FINAL_MARKERS_FOR_EACH_CLUSTER.',
                                skiprows=3)
cluster_markers = cluster_markers.rename(columns={"Unnamed: 0": 'gene_symbol'})

# Remove any rows with NA because those are all header rows
print(cluster_markers.shape)
cluster_markers = cluster_markers.dropna(subset=['gene_symbol', 'cluster #'])
print(cluster_markers.shape)
cluster_markers['cluster #'] = cluster_markers['cluster #'].astype(int)
cluster_markers.head()

(4292, 5)
(4140, 5)


,gene_symbol,myAUC,myDiff,power,cluster #
0,CALB1,0.966,3.615047,0.466,1
1,SLC4A3,0.963,3.448571,0.463,1
2,TPM3,0.965,3.151521,0.465,1
3,SEPT4,0.964,2.939258,0.464,1
4,VIM,0.944,2.937992,0.444,1


In [23]:
cluster_markers.groupby('gene_symbol').size()

gene_symbol
1500015O10RIK      1
1500016L03RIK      1
1700025G04RIK      1
1810009A15RIK      1
1810037I17RIK      1
2010107E04RIK      1
2410066E13RIK      1
2610017I09RIK      2
2900011O08RIK     11
4632428N05RIK      1
4833424O15RIK      4
4930447C04RIK      3
6330403K07RIK      7
6430548M08RIK     11
8430419L09RIK      1
A030009H04RIK     13
A330050F15RIK      1
A730046J19RIK      2
A830010M20RIK      2
A830036E02RIK      1
A930001A20RIK      1
A930011O12RIK      7
ABAT               2
ABCA8A             1
ABCB1A             1
ABCC9              1
ABCG2              1
ABHD10             1
ABHD2              1
ABHD4              1
                  ..
WFDC1              1
WFDC10             1
WIPI1              1
WLS                2
WNK4               1
WWTR1              1
YWHAB              1
YWHAB    0.253     1
YWHAG              5
YWHAH              6
YWHAQ              1
ZBTB20             1
ZCCHC12            1
ZCCHC18            4
ZCCHC18    0.7     1
ZEB2               5
Z

Sigh there are some contaminated gene names that have the AUC in them .. ahve to split the column

In [24]:
cluster_markers['gene_symbol'] = cluster_markers['gene_symbol'].str.split().str[0]
cluster_markers.groupby('gene_symbol').size()

gene_symbol
1500015O10RIK     1
1500016L03RIK     1
1700025G04RIK     1
1810009A15RIK     1
1810037I17RIK     1
2010107E04RIK     1
2410066E13RIK     1
2610017I09RIK     2
2900011O08RIK    11
4632428N05RIK     1
4833424O15RIK     4
4930447C04RIK     3
6330403K07RIK     7
6430548M08RIK    11
8430419L09RIK     1
A030009H04RIK    13
A330050F15RIK     1
A730046J19RIK     2
A830010M20RIK     2
A830036E02RIK     1
A930001A20RIK     1
A930011O12RIK     7
ABAT              2
ABCA8A            1
ABCB1A            1
ABCC9             1
ABCG2             1
ABHD10            1
ABHD2             1
ABHD4             1
                 ..
VWC2              1
VWF               1
WASF2             1
WBSCR17           1
WDR1              3
WFDC1             1
WFDC10            1
WIPI1             1
WLS               2
WNK4              1
WWTR1             1
YWHAB             2
YWHAG             5
YWHAH             6
YWHAQ             1
ZBTB20            1
ZCCHC12           1
ZCCHC18           5
ZEB2    

### Add columns to make pivotting (making a 2d matrix) easier

In [25]:
cluster_markers['cluster_name'] = 'retina_' \
    + cluster_markers['cluster #'].astype(str).str.zfill(2)
cluster_markers['in_cluster'] = True
cluster_markers.head()

,gene_symbol,myAUC,myDiff,power,cluster #,cluster_name,in_cluster
0,CALB1,0.966,3.615047,0.466,1,retina_01,True
1,SLC4A3,0.963,3.448571,0.463,1,retina_01,True
2,TPM3,0.965,3.151521,0.465,1,retina_01,True
3,SEPT4,0.964,2.939258,0.464,1,retina_01,True
4,VIM,0.944,2.937992,0.444,1,retina_01,True


In [28]:
cluster_markers.groupby('cluster_name').size()

cluster_name
retina_01    190
retina_02    174
retina_03    162
retina_04     84
retina_05    159
retina_06    156
retina_07    164
retina_08    145
retina_09    145
retina_10    120
retina_11    111
retina_12     68
retina_13    163
retina_14    127
retina_15     69
retina_16     97
retina_17     99
retina_18     76
retina_19    115
retina_20     43
retina_21     45
retina_22     51
retina_23     67
retina_24     49
retina_25     14
retina_26     87
retina_27     27
retina_28     48
retina_29     39
retina_30     60
retina_31     58
retina_32     81
retina_33     47
retina_34    147
retina_35    164
retina_36    153
retina_37    236
retina_38    147
retina_39    153
dtype: int64

In [29]:
mouse_gene_metadata = cluster_markers.pivot(index='gene_symbol', 
                                      values='in_cluster',
                                      columns='cluster_name')
mouse_gene_metadata = mouse_gene_metadata.fillna(False)
mouse_gene_metadata.head()

cluster_name,retina_01,retina_02,retina_03,retina_04,retina_05,retina_06,retina_07,retina_08,retina_09,retina_10,...,retina_30,retina_31,retina_32,retina_33,retina_34,retina_35,retina_36,retina_37,retina_38,retina_39
gene_symbol,,,,,,,,,,,,,,,,,,,,,
1500015O10RIK,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1500016L03RIK,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1700025G04RIK,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1810009A15RIK,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1810037I17RIK,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


## Pairwise markers

These are the genes that distinguish sets of cells from each other.



In [30]:
filename = os.path.join(input_folder, 'mmc4_v2.xlsx')

pairwise_markers = pd.read_excel(filename,
                                sheetname='PAIRWISE_MARKERS_FINAL.txt')
print(pairwise_markers.shape)
pairwise_markers.head()

(193620, 3)


,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Clusters,1+2,NaN
1,NaN,myP,myDiff
2,VIM,0.000000e+00,3.399495
3,NDRG1,0.000000e+00,3.258577
4,SEPT4,0.000000e+00,2.984931


This is a mess to clean ... each section is headed by the cluster numbers that are compared and have to iterate through every time you see "Clusters".

Don't need this for now

## Get cell cycle genes

In [31]:
filename = os.path.join(input_folder, 'mmc2.xlsx')
mmc2 = pd.read_excel(filename)
mmc2.head()

,cluster,human gene,cluster.1,mouse gene,Unnamed: 4,Intersection,Unnamed: 6,Unnamed: 7
0,1.0,CCNE2,1,Shmt1,NaN,All genes,novel genes,annotation
1,1.0,CDC6,1,Zmym1,NaN,ACTB,ACTB,NaN
2,1.0,CLSPN,1,Meaf6,NaN,AKIRIN2,ARHGAP11A,NaN
3,1.0,DTL,1,Usp37,NaN,ANLN,ARL6IP6,NaN
4,1.0,MCM3,1,Msh6,NaN,ANP32E,ARPC2,NaN


This is a mix of both human and mouse data ... going to use just hte mouse ones for now

In [32]:
mouse_genes = mmc2[['cluster.1', 'mouse gene']]
mouse_genes['cluster.1'] = 'cellcycle_' + mouse_genes['cluster.1'].astype(str).str.zfill(2)
mouse_genes = mouse_genes.rename(columns={'cluster.1': 'cluster_name'})
mouse_genes['in_cluster'] = True
mouse_genes.head()

/Users/olgabot/anaconda3/envs/macosko2015-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,cluster_name,mouse gene,in_cluster
0,cellcycle_01,Shmt1,True
1,cellcycle_01,Zmym1,True
2,cellcycle_01,Meaf6,True
3,cellcycle_01,Usp37,True
4,cellcycle_01,Msh6,True


In [33]:
csv = os.path.join(data_folder, 'cellcycle_genes_tidy.csv')
mouse_genes.to_csv(csv)
! ls -lh $csv
! head $csv

-rw-r--r--  1 olgabot  staff    21K Jun 26 11:22 ../data/04_extract_data_from_supplementary_excel_files/cellcycle_genes_tidy.csv
,cluster_name,mouse gene,in_cluster
0,cellcycle_01,    Shmt1,True
1,cellcycle_01,    Zmym1,True
2,cellcycle_01,    Meaf6,True
3,cellcycle_01,    Usp37,True
4,cellcycle_01,    Msh6,True
5,cellcycle_01,    Rbbp4,True
6,cellcycle_01,    Bri3bp,True
7,cellcycle_01,    Rrp8,True
8,cellcycle_01,    Mb21d1,True


In [34]:
cellcycle = mouse_genes.pivot(index='mouse gene', 
                              columns='cluster_name', 
                              values='in_cluster')
cellcycle = cellcycle.fillna(False)
cellcycle.head()

cluster_name,cellcycle_01,cellcycle_02,cellcycle_03,cellcycle_04,cellcycle_05,cellcycle_06,cellcycle_07,cellcycle_08
mouse gene,,,,,,,,
Aaas,False,False,False,False,True,False,False,False
Acat2,False,False,True,False,False,False,False,False
Acot9,False,False,False,False,False,False,True,False
Actb,False,False,False,False,False,False,False,True
Adar,False,False,False,False,False,False,False,True


### Add to growing mouse metadata list

In [35]:
mouse_gene_metadata = pd.concat([mouse_gene_metadata, cellcycle], axis=1)
print(mouse_gene_metadata.shape)
mouse_gene_metadata.head()

(2007, 47)


cluster_name,retina_01,retina_02,retina_03,retina_04,retina_05,retina_06,retina_07,retina_08,retina_09,retina_10,...,retina_38,retina_39,cellcycle_01,cellcycle_02,cellcycle_03,cellcycle_04,cellcycle_05,cellcycle_06,cellcycle_07,cellcycle_08
Aaas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,True,False,False,False
Acat2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,True,False,False,False,False,False
Acot9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,False,True,False
Actb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,False,False,True
Adar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,False,False,True


In [36]:
mouse_gene_metadata = mouse_gene_metadata.fillna(False)
mouse_gene_metadata.head()

cluster_name,retina_01,retina_02,retina_03,retina_04,retina_05,retina_06,retina_07,retina_08,retina_09,retina_10,...,retina_38,retina_39,cellcycle_01,cellcycle_02,cellcycle_03,cellcycle_04,cellcycle_05,cellcycle_06,cellcycle_07,cellcycle_08
Aaas,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
Acat2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
Acot9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
Actb,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
Adar,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


### Add index name of gene symbol

In [37]:
mouse_gene_metadata.index.name = 'gene_symbol'

In [38]:
mouse_gene_metadata.head()

cluster_name,retina_01,retina_02,retina_03,retina_04,retina_05,retina_06,retina_07,retina_08,retina_09,retina_10,...,retina_38,retina_39,cellcycle_01,cellcycle_02,cellcycle_03,cellcycle_04,cellcycle_05,cellcycle_06,cellcycle_07,cellcycle_08
gene_symbol,,,,,,,,,,,,,,,,,,,,,
Aaas,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
Acat2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
Acot9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
Actb,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
Adar,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


## Get annotated cell cycle genes

In [39]:
filename = os.path.join(input_folder, 'mmc2.xlsx')
mmc2 = pd.read_excel(filename, sheetname='Gene Sets Used in Analysis')
mmc2.head()

,G1/S,S,G2/M,M,M/G1
0,ACD,ABCC5,ANLN,AHI1,AGFG1
1,ACYP1,ABHD10,AP3D1,AKIRIN2,AGPAT3
2,ADAMTS1,ANKRD18A,ARHGAP19,ANKRD40,AKAP13
3,ANKRD10,ASF1B,ARL4A,ANLN,AMD1
4,APEX2,ATAD2,ARMC1,ANP32B,ANP32E


## Write the gene metadata to csv

In [40]:
data_folder

'../data/04_extract_data_from_supplementary_excel_files'

In [41]:

csv = os.path.join(data_folder, 'mouse_gene_metadata.csv')
mouse_gene_metadata.to_csv(csv)
! head $csv

gene_symbol,retina_01,retina_02,retina_03,retina_04,retina_05,retina_06,retina_07,retina_08,retina_09,retina_10,retina_11,retina_12,retina_13,retina_14,retina_15,retina_16,retina_17,retina_18,retina_19,retina_20,retina_21,retina_22,retina_23,retina_24,retina_25,retina_26,retina_27,retina_28,retina_29,retina_30,retina_31,retina_32,retina_33,retina_34,retina_35,retina_36,retina_37,retina_38,retina_39,cellcycle_01,cellcycle_02,cellcycle_03,cellcycle_04,cellcycle_05,cellcycle_06,cellcycle_07,cellcycle_08
    Aaas,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
    Acat2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa